In [6]:
import pandas as pd
import string
import matplotlib.pyplot as plt
import nltk
import spacy
import torch

from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk import pos_tag
from nltk import FreqDist
from wordcloud import WordCloud
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
from transformers import pipeline

nltk.download('punkt')

c:\Users\camd1\miniconda3\envs\textmining2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\camd1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from collections import Counter

# Sample documents
documents = [
    "My name is Thien.",
    "My friend's name is Marcus.",
    "We live together.",
    "Miwa sometimes visits us.",
    "Do you want to visit us?",
]

# Step 1: Tokenization
tokens = [doc.split() for doc in documents]

# Step 2: Building Vocabulary
vocabulary = set(word for doc in tokens for word in doc)

# Step 3: Vectorization
def get_bow_vector(doc, vocab):
    word_counts = Counter(doc)
    bow_vector = [word_counts[word] if word in word_counts else 0 for word in vocab]
    return bow_vector

# Create Bag-of-Words vectors for each document
bow_vectors = [get_bow_vector(doc, vocabulary) for doc in tokens]

# Display the Bag-of-Words vectors
for i, vec in enumerate(bow_vectors):
    print(f"Document {i+1}:", vec)


Document 1: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
Document 2: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
Document 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Document 4: [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Document 5: [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
import pandas as pd
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
pipeline = TokenClassificationPipeline(model=model, tokenizer=tokenizer)

def extract_nouns(tokens):
    nouns = []
    pos_nouns = pipeline(tokens)
    for result in pos_nouns:
        if result['entity'] == 'NN':
            nouns.append(result['word'])
    return nouns

# Create an example DataFrame
example_df = pd.DataFrame({'dialogue': ['I would like to run a car over a turtle.', 'Oh my god']})

# Apply the extract_nouns function to create the 'nouns' column
example_df['nouns'] = example_df['dialogue'].apply(extract_nouns)
example_df['num_nouns'] = example_df['dialogue'].apply(lambda x: len(extract_nouns(x)))

# Print the example DataFrame
print(example_df)

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                   dialogue              nouns  num_nouns
0  I would like to run a car over a turtle.  [car, tur, ##tle]          3
1                                 Oh my god              [god]          1


In [21]:
outputs = pipeline("I would like to run a car over a turtle.")
print(outputs)

[{'entity': 'PRP', 'score': 0.99967754, 'index': 1, 'word': 'I', 'start': 0, 'end': 1}, {'entity': 'MD', 'score': 0.999617, 'index': 2, 'word': 'would', 'start': 2, 'end': 7}, {'entity': 'VB', 'score': 0.99917704, 'index': 3, 'word': 'like', 'start': 8, 'end': 12}, {'entity': 'TO', 'score': 0.9998995, 'index': 4, 'word': 'to', 'start': 13, 'end': 15}, {'entity': 'VB', 'score': 0.99969435, 'index': 5, 'word': 'run', 'start': 16, 'end': 19}, {'entity': 'DT', 'score': 0.9998702, 'index': 6, 'word': 'a', 'start': 20, 'end': 21}, {'entity': 'NN', 'score': 0.9997905, 'index': 7, 'word': 'car', 'start': 22, 'end': 25}, {'entity': 'IN', 'score': 0.9996146, 'index': 8, 'word': 'over', 'start': 26, 'end': 30}, {'entity': 'DT', 'score': 0.99986935, 'index': 9, 'word': 'a', 'start': 31, 'end': 32}, {'entity': 'NN', 'score': 0.9996551, 'index': 10, 'word': 'tur', 'start': 33, 'end': 36}, {'entity': 'NN', 'score': 0.99923944, 'index': 11, 'word': '##tle', 'start': 36, 'end': 39}, {'entity': '.', 'sc

In [1]:
import pandas as pd

# Sample DataFrame
data = {
    'A': [25, 30, 15, 40],
    'B': [35, 20, 10, 50],
    'C': [45, 10, 30, 60]
}

df = pd.DataFrame(data)

# Find the 10 highest values and get their column names
highest_values = df.stack().nlargest(10).index.get_level_values(1)

# Add the column names of the 10 highest values to a set in result['words']
result = {'words': set(highest_values)}

print(result)


{'words': {'C', 'B', 'A'}}


In [ ]:
# Create sorted bag of words
def create_bow(corpus):    
    bow = set()
    for doc in corpus:
        bow = bow.union(set(doc))
    return sorted(bow)

# Find 10 most important words using TF-IDF
def imp_tf_idf(corpus, bow):
    # Calculate term frequency (TF)
    n_docs = len(corpus)         # Number of documents in the corpus
    n_words_set = len(bow)       # Number of unique words in the corpus

    df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), columns=bow)

    for i in range(n_docs):
        n_words = len(corpus[i])    # Number of words in the document
        for w in corpus[i]:
            df_tf[w][i] = df_tf[w][i] + (1 / n_words)
        
    # Calculate Inverse Document Frequency (IDF)
    idf = {}
    for w in bow:
        k = 0    # number of documents in the corpus that contain this word

        for i in range(n_docs):
            if w in corpus[i]:
                k += 1
                
        idf[w] =  np.log10(n_docs / k)

    # Find most important words in the corpus
    df_tf_idf = df_tf.copy()

    for w in bow:
        for i in range(n_docs):
            df_tf_idf[w][i] = df_tf[w][i] * idf[w]
    
    highest_values = df_tf_idf.stack().nlargest(10).index.get_level_values(1).tolist()
    return highest_values

In [ ]:
import numpy as np

# Create sorted bag of words
def create_bow(corpus):    
    bow = set()
    for doc in corpus:
        bow = bow.union(set(doc))
    return sorted(bow)

# Find 10 most important words using TF-IDF
def imp_tf_idf(corpus, bow):
    # Calculate term frequency (TF)
    n_docs = len(corpus)         # Number of documents in the corpus
    n_words_set = len(bow)       # Number of unique words in the corpus

    # Initialize TF matrix
    tf_matrix = np.zeros((n_docs, n_words_set))

    for i in range(n_docs):
        n_words = len(corpus[i])    # Number of words in the document
        for w in corpus[i]:
            word_index = bow.index(w)
            tf_matrix[i][word_index] += 1 / n_words
        
    # Calculate Inverse Document Frequency (IDF)
    idf = np.zeros(n_words_set)
    for idx, w in enumerate(bow):
        k = sum(1 for doc in corpus if w in doc)
        idf[idx] = np.log10(n_docs / k) if k != 0 else 0

    # Calculate TF-IDF matrix
    tf_idf_matrix = tf_matrix * idf

    # Find most important words in the corpus
    flat_tf_idf = tf_idf_matrix.flatten()
    highest_indices = np.argpartition(flat_tf_idf, -10)[-10:]
    highest_values = [bow[i // n_docs] for i in highest_indices]
    return highest_values
